# Mapas

### Crear archivo para Mapa en masivo

In [7]:
from google.cloud import bigquery
import pandas as pd

# Option 1: Using a service account key file path
client = bigquery.Client.from_service_account_json(r"D:\GitLab\pe-pacasmayo-analytics-gcp-19997eeb928e.json")

# 2. Define el ID de tu proyecto, dataset y tabla.
# Reemplaza estos valores con los tuyos.
project_id = "pe-pacasmayo-analytics-gcp"
dataset_id = "prod_dwh_comercial_cpsaa"
table_id = "comercial_lcm_zona_comercial"

# 3. Construye la referencia completa a la tabla.
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# 4. Define tu consulta SQL.
# En este caso, seleccionamos todo de la tabla.
sql_query = f"""
    SELECT *
    FROM `{table_ref}`

"""

# 5. Ejecuta la consulta y convierte el resultado a un DataFrame de Pandas.
# El método .to_dataframe() se encarga de todo el trabajo pesado por ti.
try:
    df = client.query(sql_query).to_dataframe()

except Exception as e:
    print(f"Ocurrió un error: {e}")

#df = df[(df['idzonacomercial']!=0)&(df['idzonacomercial']!=10)]

In [9]:
import datetime
df_filter = df[(df["fecha_fin_vigencia"] == datetime.date(2025, 8, 31))&(df["idzonacomercial"] != 0)].copy()
df_filter

,idzonacomercial,deszonacomercial,codigozonacomercial,geometry,codisicom_isicom_asociado,FECRUTINA,FECCARGA,TIPFRECUENCIA,FLGELIMINADO,FECELIMINACION,fecha_inicio_vigencia,fecha_fin_vigencia
80,100.000000000,50002965 - INVERSIONES BEATRIZ,100,"{""type"":""Polygon"",""coordinates"":[[[-80.8286801...",201900000023.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
81,101.000000000,50009197 - E&L CASTILLO,101,"{""type"":""Polygon"",""coordinates"":[[[-80.3454219...",201900012371.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
82,102.000000000,1800005622 - MI KARINA,102,"{""type"":""Polygon"",""coordinates"":[[[-80.6948849...",202000022215.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
83,103.000000000,50009981 - PEYMAR,103,"{""type"":""Polygon"",""coordinates"":[[[-80.8120057...",201900012384.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
84,104.000000000,50013838 - NORPERU 1,104,"{""type"":""Polygon"",""coordinates"":[[[-79.8679465...",201900006688.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...
149,95.000000000,NERUSA,95,"{""type"":""Polygon"",""coordinates"":[[[-80.6256211...",201900002336.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
150,96.000000000,BAISTA 1,96,"{""type"":""Polygon"",""coordinates"":[[[-79.8546116...",201900016521.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
151,97.000000000,BAISTA 2,97,"{""type"":""Polygon"",""coordinates"":[[[-79.844672,...",201900016521.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31
152,98.000000000,11402283 - ARENERA JAEN,98,"{""type"":""Polygon"",""coordinates"":[[[-80.6766348...",201900006018.000000000,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2025-05-01,2025-08-31


In [72]:
###Validar que las zonas esten cerradas

#json.loads(df_filter["geometry"][154])['coordinates'][0]#[0] ##[-80.1717081, -5.0887526] Validar que las zonas esten cerradas
#json.loads(df_filter["geometry"][154])['coordinates'][0].pop() ##[-80.1717081, -5.0887526]

df_filter["coor1"] = df_filter["geometry"].apply(lambda x: json.loads(x)['coordinates'][0][0])
df_filter["coor2"] = df_filter["geometry"].apply(lambda x: json.loads(x)['coordinates'][0].pop())
df_filter["validacion"] = df_filter["coor1"] == df_filter["coor2"]
df_filter[df_filter["validacion"]==False]

,idzonacomercial,deszonacomercial,codigozonacomercial,geometry,codisicom_isicom_asociado,FECRUTINA,FECCARGA,TIPFRECUENCIA,FLGELIMINADO,FECELIMINACION,fecha_inicio_vigencia,fecha_fin_vigencia,coor1,coor2,validacion


In [10]:

import folium
import json
import random

# 2. Crear el mapa base
# Centramos el mapa en una ubicación inicial (ej. Lima, Perú). 
# Ajusta la latitud y longitud si tus datos están en otra parte.
mapa = folium.Map(location=[-12.08, -77.04], zoom_start=12)

# 3. Iterar sobre el DataFrame para agregar cada polígono
for col, fila in df_filter.iterrows():
    try:
        print(fila['deszonacomercial'])
        random_color = f'#{random.randint(0, 0xFFFFFF):06x}'
        # Extraer el nombre de la zona
        nombre_zona = fila['deszonacomercial']
        
        # Cargar la cadena de texto JSON a un diccionario de Python
        geom_json = json.loads(fila['geometry'])
        
        # Extraer las coordenadas. La estructura es [lon, lat]
        # Nota: El JSON puede tener varias listas anidadas, por eso el [0]
        coordenadas_raw = geom_json['coordinates'][0]
        
        # Folium necesita las coordenadas en formato [lat, lon]
        # Creamos una nueva lista invirtiendo el orden de cada par.
        coordenadas_folium = [[lat, lon] for lon, lat in coordenadas_raw]

        # 4. Agregar el polígono al mapa
        folium.Polygon(
            locations=coordenadas_folium,
            color=random_color,         # Color del borde
            fill=True,            # Rellenar el polígono
            fill_color=random_color,    # Color del relleno
            fill_opacity=0.7,     # Transparencia del relleno
            tooltip=nombre_zona,   # Texto que aparece al pasar el mouse
        ).add_to(mapa)

    except (json.JSONDecodeError, TypeError, KeyError, IndexError) as e:
        print(f"No se pudo procesar la fila para '{fila['deszonacomercial']}': {e}")
        # Esta línea es útil para depurar si algún JSON está mal formado
mapa
# 5. Guardar el mapa en un archivo HTML
mapa.save("mapa_de_zonas.html")

print("¡Mapa 'mapa_de_zonas.html' generado exitosamente!")
print("Abre este archivo en tu navegador para ver el resultado.")

50002965 - INVERSIONES BEATRIZ
50009197 - E&L CASTILLO
1800005622 - MI KARINA
50009981 - PEYMAR
50013838 - NORPERU 1
50013838 - NORPERU 2
50013838 - NORPERU 3
1800016491 - JABIAL
1800003655 - SOLUCIONES ACERO
1800014418 - YENI GARCIA
50007406 - SALVADOR
50003356 - EMMANUEL 1
50003356 - EMMANUEL 2
50003356 - EMMANUEL 3
1800011467 - ANGIE LEON
1800013053 - GRUPO JEHOVA
1800003573 - CIMIENTOS EL CID
1800021619 - REPRESENTACIONES Y SERVICIOS EL CLAVITOS.R.L.
1800001817 - CASA DEL CEMENTO
50014099 - EL OVALO
50013608 - CLEINER COLONIA
1800005267 - CONSTRUPLAZA
50009017 - MI FLORITA
50013913 - GRUPO ROJO
1800019331 - MOVAL GROUP
50100865 - FERRETERIA MORENO
50014427 - FERRETERIA ESPERANZA
50008418 - SANTA MARIA 1
50008380 - ESMERALDA CORAL
50100839 - JUNIOR TORRES
50011932 - KARINA TORRES
1800013017 - MEGACONSTRUCCIONES MOSTACERO
50102195 - DIMACON
50008418 - SANTA MARIA 2
1800011256 - RONAL LUERA
50003080 - ARNALDO VEGA
50100377 - FUTURA EVOLUTION
50009566 - CUBA
50100081 - ROLY VILLACORTA


### Script para ver 1 geocerca en folium por puntos

### prompt: Quiero que del siguiente formato geo_json, me devuelvas una lista de tuplas, donde se haya invertido el orden de latitud y longuitud de cada punto.

In [6]:
import folium

# Lista de coordenadas para el polígono
polygon_points_5 = [
(-5.0926646, -80.1539263),
(-5.0888174, -80.1537547),
(-5.0889061, -80.1538008),
(-5.0821529, -80.1535752),
(-5.0705985, -80.1432215),
(-5.0558932, -80.1242529),
(-5.0497374, -80.1228796),
(-5.0446076, -80.1275145),
(-5.045377, -80.1325785),
(-5.015623, -80.1303469),
(-5.0133145, -80.1763522),
(-5.0277642, -80.1929175),
(-5.0404181, -80.2121436),
(-5.045548, -80.2066504),
(-5.0475145, -80.19369),
(-5.0390502, -80.1887118),
(-5.0450351, -80.1782404),
(-5.048455, -80.1684557),
(-5.0584669, -80.1621661),
(-5.0608608, -80.1534113),
(-5.0665035, -80.152553),
(-5.081636, -80.1577027),
(-5.0858251, -80.1716075),
(-5.0906983, -80.1714358),
(-5.0940325, -80.1693758),
(-5.0911258, -80.1626811),
(-5.0909547, -80.1582178),
(-5.0912967, -80.1550421),
(-5.0926646, -80.1539263)
]

# --- MEJORA: Centrar el mapa automáticamente en el polígono ---
# Calculamos el punto central de todos los puntos para centrar el mapa.
avg_lat = sum(p[0] for p in polygon_points_5) / len(polygon_points_5)
avg_lon = sum(p[1] for p in polygon_points_5) / len(polygon_points_5)
map_center = [avg_lat, avg_lon]

# Creamos el mapa centrado en el polígono
m = folium.Map(location=map_center, zoom_start=14)

# Agregamos el polígono al mapa (tu código original)
folium.Polygon(
    locations=polygon_points_5,
    color='red',          # Color del borde
    fill=True,
    fill_color='red',     # Color de relleno
    fill_opacity=0.2,     # Opacidad del relleno
    tooltip="1800015855 - V&M"
).add_to(m)

# --- MODIFICACIÓN: Agregar un marcador con número correlativo para cada punto ---
# Usamos enumerate(..., start=1) para obtener un índice (idx) que empieza en 1.
for idx, point in enumerate(polygon_points_5, start=1):
    folium.Marker(
        location=point,
        # En el popup, mostramos el número del punto y sus coordenadas.
        popup=f"<b>Punto {idx}</b><br>Coordenadas:<br>{point[0]}<br>{point[1]}",
        # En el tooltip, mostramos el número del punto para una identificación rápida.
        tooltip=f"Punto {idx}"
    ).add_to(m)
# --- FIN DE LA MODIFICACIÓN ---

# Agregamos un control de capas (opcional, pero útil)
folium.LayerControl().add_to(m)

# -----------------------------------------------------------------
# --- INICIO DE LA MODIFICACIÓN: AGREGAR PUNTO ESPECÍFICO ---
# -----------------------------------------------------------------
# 1. Define las coordenadas de tu punto. ¡Modifica estos valores!
punto_especifico = (-5.09046996, -80.15951995)

# 2. Crea el marcador para ese punto
folium.Marker(
    location=punto_especifico,
    popup="<b>Mi Punto Específico</b><br>¡Ubicación importante!",
    tooltip="Clic para ver detalles",
    # Personaliza el ícono para que se destaque
    icon=folium.Icon(color='blue', icon='star')
).add_to(m)
# -----------------------------------------------------------------
# --- FIN DE LA MODIFICACIÓN ---
# -----------------------------------------------------------------

# Guardamos el mapa en un archivo HTML
output_filename = "mapa_con_poligono_y_puntos_numerados.html"
m.save(output_filename)

print(f"Mapa guardado como '{output_filename}'")

Mapa guardado como 'mapa_con_poligono_y_puntos_numerados.html'


In [4]:
import folium

# Lista de coordenadas para el polígono
polygon_points_5 = [
    (-5.0926646, -80.1539263), (-5.0888174, -80.1537547),
    (-5.0889061, -80.1538008), (-5.0821529, -80.1535752),
    (-5.0705985, -80.1432215), (-5.0558932, -80.1242529),
    (-5.0497374, -80.1228796), (-5.0446076, -80.1275145),
    (-5.045377, -80.1325785), (-5.015623, -80.1303469),
    (-5.0133145, -80.1763522), (-5.0277642, -80.1929175),
    (-5.0404181, -80.2121436), (-5.045548, -80.2066504),
    (-5.0475145, -80.19369), (-5.0390502, -80.1887118),
    (-5.0450351, -80.1782404), (-5.048455, -80.1684557),
    (-5.0584669, -80.1621661), (-5.0608608, -80.1534113),
    (-5.0665035, -80.152553), (-5.081636, -80.1577027),
    (-5.0858251, -80.1716075), (-5.0906983, -80.1714358),
    (-5.0940325, -80.1693758), (-5.0911258, -80.1626811),
    (-5.0909547, -80.1582178), (-5.0912967, -80.1550421),
    (-5.0926646, -80.1539263)
]

# --- Centrar el mapa automáticamente en el polígono ---
avg_lat = sum(p[0] for p in polygon_points_5) / len(polygon_points_5)
avg_lon = sum(p[1] for p in polygon_points_5) / len(polygon_points_5)
map_center = [avg_lat, avg_lon]

# Creamos el mapa centrado
m = folium.Map(location=map_center, zoom_start=14)

# --- Agregamos el polígono al mapa ---
folium.Polygon(
    locations=polygon_points_5,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.2,
    tooltip="1800015855 - V&M"
).add_to(m)

# --- Agregar un marcador para cada punto del polígono ---
for idx, point in enumerate(polygon_points_5, start=1):
    folium.Marker(
        location=point,
        popup=f"<b>Punto {idx}</b><br>Coordenadas:<br>{point[0]}<br>{point[1]}",
        tooltip=f"Punto {idx}"
    ).add_to(m)

# -----------------------------------------------------------------
# --- INICIO DE LA MODIFICACIÓN: AGREGAR PUNTO ESPECÍFICO ---
# -----------------------------------------------------------------
# 1. Define las coordenadas de tu punto. ¡Modifica estos valores!
punto_especifico = (-5.06, -80.18)

# 2. Crea el marcador para ese punto
folium.Marker(
    location=punto_especifico,
    popup="<b>Mi Punto Específico</b><br>¡Ubicación importante!",
    tooltip="Clic para ver detalles",
    # Personaliza el ícono para que se destaque
    icon=folium.Icon(color='blue', icon='star')
).add_to(m)
# -----------------------------------------------------------------
# --- FIN DE LA MODIFICACIÓN ---
# -----------------------------------------------------------------

# Agregamos un control de capas
folium.LayerControl().add_to(m)

# Guardamos el mapa en un archivo HTML
output_filename = "mapa_con_punto_especifico.html"
m.save(output_filename)

print(f"Mapa guardado como '{output_filename}'")

Mapa guardado como 'mapa_con_punto_especifico.html'


### Colocar georeferencia en forma folium

In [1]:
import json

geo_json_str = '{"type":"Polygon","coordinates":[[[-80.1539263,-5.0926646],[-80.1537547,-5.0888174],[-80.1538008,-5.0889061],[-80.1535752,-5.0821529],[-80.1432215,-5.0705985],[-80.1242529,-5.0558932],[-80.1228796,-5.0497374],[-80.1275145,-5.0446076],[-80.1325785,-5.045377],[-80.1303469,-5.015623],[-80.1763522,-5.0133145],[-80.1929175,-5.0277642],[-80.2121436,-5.0404181],[-80.2066504,-5.045548],[-80.19369,-5.0475145],[-80.1887118,-5.0390502],[-80.1782404,-5.0450351],[-80.1684557,-5.048455],[-80.1621661,-5.0584669],[-80.1534113,-5.0608608],[-80.152553,-5.0665035],[-80.1577027,-5.081636],[-80.1716075,-5.0858251],[-80.1714358,-5.0906983],[-80.1693758,-5.0940325],[-80.1626811,-5.0911258],[-80.1582178,-5.0909547],[-80.1550421,-5.0912967],[-80.1539263,-5.0926646]]]}'

geo_json = json.loads(geo_json_str)

# Extraer la lista de coordenadas
coordinates = geo_json['coordinates'][0]

# Invertir el orden de cada par de coordenadas
inverted_coordinates = [(lat, lon) for lon, lat in coordinates]

# Unir cada tupla con un carácter de nueva línea
output_str = ",\n".join(map(str, inverted_coordinates))

print(output_str)

(-5.0926646, -80.1539263),
(-5.0888174, -80.1537547),
(-5.0889061, -80.1538008),
(-5.0821529, -80.1535752),
(-5.0705985, -80.1432215),
(-5.0558932, -80.1242529),
(-5.0497374, -80.1228796),
(-5.0446076, -80.1275145),
(-5.045377, -80.1325785),
(-5.015623, -80.1303469),
(-5.0133145, -80.1763522),
(-5.0277642, -80.1929175),
(-5.0404181, -80.2121436),
(-5.045548, -80.2066504),
(-5.0475145, -80.19369),
(-5.0390502, -80.1887118),
(-5.0450351, -80.1782404),
(-5.048455, -80.1684557),
(-5.0584669, -80.1621661),
(-5.0608608, -80.1534113),
(-5.0665035, -80.152553),
(-5.081636, -80.1577027),
(-5.0858251, -80.1716075),
(-5.0906983, -80.1714358),
(-5.0940325, -80.1693758),
(-5.0911258, -80.1626811),
(-5.0909547, -80.1582178),
(-5.0912967, -80.1550421),
(-5.0926646, -80.1539263)


### Script para Leer archivos de Apache Beam

In [11]:
import pandas as pd
import ast # Abstract Syntax Tree, necesario para la evaluación segura

# 1. Especifica la ruta a tu archivo .txt
#    Reemplaza esto con la ruta real de tu archivo.
file_path = r'D:\GitLabBQ\proyecto-dataflow-sapbq\enc_vis-00000-of-00001 - copia.txtt'

# 2. Lee el archivo línea por línea y conviértelo a un DataFrame
data_list = []
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Elimina espacios en blanco o saltos de línea al inicio/final
            stripped_line = line.strip() 
            if stripped_line: # Procesa la línea solo si no está vacía
                try:
                    # ast.literal_eval convierte de forma segura un string
                    # que parece un objeto de Python (como un dict) a ese objeto.
                    record_dict = ast.literal_eval(stripped_line)
                    data_list.append(record_dict)
                except (ValueError, SyntaxError) as e:
                    print(f"Advertencia: Se omitió una línea por un error de formato: {e}")
                    print(f"Línea con problemas: {stripped_line[:50]}...") # Muestra parte de la línea

    # 3. Crea el DataFrame de pandas a partir de la lista de diccionarios
    df = pd.DataFrame(data_list)

    # 4. Muestra los resultados para verificar
    print("¡Archivo leído exitosamente!")
    print("\nPrimeras 5 filas del DataFrame:")
    print(df.head())

    print("\nInformación del DataFrame:")
    df.info()

except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo en la ruta especificada: {file_path}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Error: No se pudo encontrar el archivo en la ruta especificada: D:\GitLabBQ\proyecto-dataflow-sapbq\enc_vis-00000-of-00001 - copia.txtt


In [ ]:
import pandas as pd
import ast
import json

data_list = []
file_path = r'D:\GitLabBQ\proyecto-dataflow-sapbq\enc_vis-00000-of-00001 - copia.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        x = line
        exit

x

In [ ]:
json.dumps(x)

In [ ]:
import apache_beam as beam

with beam.Pipeline(runner='DirectRunner') as p:
    dictionaries = (
      p
      | 'Create PCollection' >> beam.io.ReadFromText(r'D:\GitLabBQ\proyecto-dataflow-sapbq\enc_vis-00000-of-00001 - copia.txt')
      # 2. Aplicamos una transformación. Usamos beam.Map para aplicar la función
      #    ast.literal_eval a cada elemento (en este caso, solo hay uno).
      #| 'Parse String to Dict' >> beam.Map(ast.literal_eval)
      |'printeando'>> beam.Map(lambda x: x["iddia"])
      |'printeando'>> beam.Map(print)
  )


In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

penguins = sns.load_dataset("penguins")

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=penguins, kind="bar",
    x="species X", y="body_mass_g", hue="sex",
    errorbar="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("", "Body mass (g)")
g.legend.set_title("")

# Scripts para cargar las zonas en archivos geojson en cloudstorage con una metadata geojson para lectura directa.

In [22]:
from google.cloud import bigquery
import pandas as pd

# Option 1: Using a service account key file path
client = bigquery.Client.from_service_account_json(r"D:\GitLab\pe-pacasmayo-analytics-gcp-19997eeb928e.json")

# 2. Define el ID de tu proyecto, dataset y tabla.
# Reemplaza estos valores con los tuyos.
project_id = "pe-pacasmayo-analytics-gcp"
dataset_id = "prod_dwh_comercial_cpsaa"
table_id = "comercial_lcm_zona_comercial"

# 3. Construye la referencia completa a la tabla.
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# 4. Define tu consulta SQL.
# En este caso, seleccionamos todo de la tabla.
sql_query = f"""
    SELECT *
    FROM `{table_ref}`

"""

# 5. Ejecuta la consulta y convierte el resultado a un DataFrame de Pandas.
# El método .to_dataframe() se encarga de todo el trabajo pesado por ti.
try:
    df = client.query(sql_query).to_dataframe()

except Exception as e:
    print(f"Ocurrió un error: {e}")

#df = df[(df['idzonacomercial']!=0)&(df['idzonacomercial']!=10)]

In [26]:
import json

def to_geojson_feature(row):
    geom = json.loads(row["geometry"])
    feature = {"type": "Feature",
                        "properties": {"idzonacomercial_calc": row["idzonacomercial"]},
        "geometry": geom    }
    return feature

# Lista de todas las combinaciones unicas posibles de codigo isicom + fecha_fin_vigencia
df["codisicom_isicom_asociado"] = df["codisicom_isicom_asociado"].fillna(0)
df["codisicom_isicom_asociado"] = df["codisicom_isicom_asociado"].astype(str)
df["codisicom_isicom_asociado"] = df['codisicom_isicom_asociado'].str.split('.', expand=True)[0]
df["idzonacomercial"] = df["idzonacomercial"].astype(str)
df["idzonacomercial"] = df["idzonacomercial"].str.split('.', expand=True)[0]
df["key"] = df["codisicom_isicom_asociado"] + "_" + df["fecha_fin_vigencia"].astype(str)
df["link"] = "https://raw.githubusercontent.com/fwong90/SG360/refs/heads/main/" + df["key"] + ".geojson"
df["features"] = df.apply(to_geojson_feature, axis=1).tolist()
df

,idzonacomercial,deszonacomercial,codigozonacomercial,geometry,codisicom_isicom_asociado,FECRUTINA,FECCARGA,TIPFRECUENCIA,FLGELIMINADO,FECELIMINACION,fecha_inicio_vigencia,fecha_fin_vigencia,key,link,features
0,0E-9,Sin Zona,0,"{""type"":""Polygon"",""coordinates"":[[[0,0,0]]]}",0,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2024-01-01,2025-04-30,0_2025-04-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
1,15,INVERSIONES LJ DIAZ JLO,1,"{""type"":""Polygon"",""coordinates"":[[[-79.8409632...",201900003035,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2024-01-01,2025-04-30,201900003035_2025-04-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
2,1,LADRIMART,1,"{""type"":""Polygon"",""coordinates"":[[[-78.5944166...",201900003170,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2024-01-01,2025-04-30,201900003170_2025-04-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
3,10,EL OVALO,10,"{""type"":""Polygon"",""coordinates"":[[[-78.5168837...",201900002657,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2024-01-01,2025-04-30,201900002657_2025-04-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
4,11,CONSTRUPLAZA 1,11,"{""type"":""Polygon"",""coordinates"":[[[-78.5045397...",202000020689,2025-05-19,2025-05-19 17:53:20.644916,M,False,NaT,2024-01-01,2025-04-30,202000020689_2025-04-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,229,50009981 - PEYMAR,229,"{""type"":""Polygon"",""coordinates"":[[[-80.8120057...",201900012384,2025-09-01,2025-09-01 11:48:08.488139,D,False,NaT,2025-09-01,2025-11-30,201900012384_2025-11-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
230,230,50007406 - SALVADOR,230,"{""type"":""Polygon"",""coordinates"":[[[-81.1054381...",201900010799,2025-09-01,2025-09-01 11:48:08.488139,D,False,NaT,2025-09-01,2025-11-30,201900010799_2025-11-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
231,231,1800015855 - V&M,231,"{""type"":""Polygon"",""coordinates"":[[[-80.6139843...",202000042863,2025-09-01,2025-09-01 11:48:08.488139,D,False,NaT,2025-09-01,2025-11-30,202000042863_2025-11-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."
232,232,50002193 - SAN SEBASTIAN,232,"{""type"":""Polygon"",""coordinates"":[[[-80.6768001...",201900018014,2025-09-01,2025-09-01 11:48:08.488139,D,False,NaT,2025-09-01,2025-11-30,201900018014_2025-11-30,https://raw.githubusercontent.com/fwong90/SG36...,"{'type': 'Feature', 'properties': {'idzonacome..."


In [25]:
import os

for i in df["key"].unique():
    df_temp = df[df["key"]==i]
    geojson = {
        "type": "FeatureCollection",
        "features": df_temp["features"].to_list()
    }

    ruta_carpeta = "D:/GitHub_personal/SG360" 
    nombre_archivo = i+".geojson"
    ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)
    os.makedirs(ruta_carpeta, exist_ok=True)

    print(f"Guardando archivo en: {ruta_completa}")
    # Export to .geojson file
    with open(ruta_completa, "w") as f:
        json.dump(geojson, f, indent=0)

print("¡Archivo guardado exitosamente!")

Guardando archivo en: D:/GitHub_personal/SG360\0_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900003035_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900003170_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900002657_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\202000020689_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900007827_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900017652_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900011116_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900006688_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900016521_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900020258_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\201900020253_2025-04-30.geojson
Guardando archivo en: D:/GitHub_personal/SG360\202400037051_202

In [1]:
lista = [1,2,3]

In [ ]:
str(lista)

'[1, 2, 3]'

In [2]:
import sys

In [6]:
sys.argv

['c:\\Users\\fwong\\AppData\\Local\\anaconda3\\Lib\\site-packages\\ipykernel_launcher.py',
 '--f=c:\\Users\\fwong\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3c9698856332bb6ad287652aa538683973da2053d.json']

In [50]:
import json
import pandas as pd

df = pd.read_json(r'C:\Users\fwong\Desktop\thunder-file_62a0988e.json')

In [51]:
df

,status,statusCode,timestamp,path,data
0,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '738d7ba0-c8af-4862-b92c-196640ce7019',..."
1,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '2fd08ca6-a24d-4bee-814e-a24270305bda',..."
2,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',..."
3,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',..."
4,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',..."
...,...,...,...,...,...
56465,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '84ed6a65-2844-420d-937a-d08e50fe7eb0',..."
56466,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',..."
56467,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',..."
56468,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',..."


In [52]:
df.iloc[1,4]["updatedDate"]

'2025-11-17'

In [53]:
df["updatedDate"] = df["data"].apply(lambda x: x.get("updatedDate", "-"))

In [54]:
df["code"] = df["data"].apply(lambda x: x.get("code", "-"))

In [55]:
df["updatedDate"].max()

'2025-11-17'

In [56]:
df["updatedDate"].min()

'2024-12-26'

In [57]:
df

,status,statusCode,timestamp,path,data,updatedDate,code
0,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '738d7ba0-c8af-4862-b92c-196640ce7019',...",2025-11-17,145000000136
1,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '2fd08ca6-a24d-4bee-814e-a24270305bda',...",2025-11-17,53000000574
2,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
3,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
4,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
...,...,...,...,...,...,...,...
56465,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '84ed6a65-2844-420d-937a-d08e50fe7eb0',...",2025-01-13,10000000002
56466,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-04,10000000001
56467,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-04,10000000001
56468,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-05,10000000001


In [47]:
len(df["code"].unique())

15919

In [58]:
df

,status,statusCode,timestamp,path,data,updatedDate,code
0,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '738d7ba0-c8af-4862-b92c-196640ce7019',...",2025-11-17,145000000136
1,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '2fd08ca6-a24d-4bee-814e-a24270305bda',...",2025-11-17,53000000574
2,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
3,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
4,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'd19eb664-b189-47df-a6d9-783d4d8f0b03',...",2025-11-17,105000000408
...,...,...,...,...,...,...,...
56465,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': '84ed6a65-2844-420d-937a-d08e50fe7eb0',...",2025-01-13,10000000002
56466,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-04,10000000001
56467,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-04,10000000001
56468,True,200,2025-11-18 01:02:14.594000+00:00,/bizner/report/orders,"{'id': 'f9f10897-dcaf-4d68-aa2d-6e410c239001',...",2025-01-05,10000000001


In [49]:
df[df["code"]=='105000000407']["data"][6]

{'id': 'ae887246-6201-4219-9851-c28d6afca5f2',
 'code': '105000000407',
 'status': 'PENDING_CONFIRMATION',
 'vendor': {'id': '7e50a4ca-b71a-4b18-9b51-ca67061cec92',
  'ruc': '20487625061',
  'code': '201900000047',
  'name': 'PALMANDINA S.A.C.',
  'ubigeo': '140105',
  'erpCode': None},
 'channel': None,
 'creator': None,
 'erpCode': None,
 'payment': {'name': 'EFECTIVO',
  'type': None,
  'erpCode': None,
  'provider': None,
  'transactionId': None,
  'paymentMethodCode': 'CASH'},
 'pricing': {'tax': None,
  'total': 283.9,
  'subtotal': 283.9,
  'deliveryCost': None,
  'adjustmentsTotal': None},
 'comments': None,
 'customer': {'id': '4457374d-8c66-4093-b41e-ac0d6980a1e2',
  'ruc': '10408626175',
  'code': '201900008448',
  'phone': '979650458',
  'lastName': 'Ramirez',
  'firstName': 'Walter',
  'loginCode': None,
  'businessName': 'Ferretería QUINTOS',
  'corporateEmail': 'walterpilarramirezquintos@gmail.com',
  'addressUbigeoCode': '140105'},
 'shipping': {'name': None,
  'deliver